In [1]:
import copy
from collections import OrderedDict, defaultdict

import numpy as np
import pandas as pd
from spikeextractors import RecordingExtractor, SortingExtractor
import spikeextractors as se

import spikemetrics.metrics as metrics
import spiketoolkit as st
from spikemetrics.utils import Epoch, printProgressBar
from spiketoolkit.curation.thresholdcurator import ThresholdCurator
import spiketoolkit as st

from spiketoolkit.validation.validation_tools import (
    get_all_metric_data,
    get_amplitude_metric_data,
    get_pca_metric_data,
    get_spike_times_metrics_data,
)

In [2]:
from spiketoolkit.validation.amplitude_cutoff import AmplitudeCutoff
from spiketoolkit.validation.metric_data import MetricData

In [3]:
rec, sort = se.example_datasets.toy_example(duration=10, num_channels=4)
mc = st.validation.MetricCalculator(sort, rec)
mc.compute_all_metric_data()

_ = mc.compute_metrics()
metric_dict = mc.get_metrics_dict()
assert type(mc._recording._recording).__name__ == 'BandpassFilterRecording' #check if filter by default
assert 'firing_rate' in metric_dict.keys()
assert 'num_spikes' in metric_dict.keys()
assert 'isi_viol' in metric_dict.keys()
assert 'presence_ratio' in metric_dict.keys()
assert 'amplitude_cutoff' in metric_dict.keys()
assert 'max_drift' in metric_dict.keys()
assert 'cumulative_drift' in metric_dict.keys()
assert 'silhouette_score' in metric_dict.keys()
assert 'isolation_distance' in metric_dict.keys()
assert 'l_ratio' in metric_dict.keys()
assert 'd_prime' in metric_dict.keys()
assert 'nn_hit_rate' in metric_dict.keys()
assert 'nn_miss_rate' in metric_dict.keys()
assert 'snr' in metric_dict.keys()
assert mc.is_filtered()

In [4]:
metric_dict['amplitude_cutoff']

[array([0.31559924, 0.01078261, 0.01078261, 0.03735329, 0.0103513 ,
        0.01078261, 0.5       , 0.5       , 0.0103513 , 0.5       ])]

In [13]:
mc = MetricData(sorting=sort, recording=rec, apply_filter=False)
mc.compute_amplitudes()
ac = AmplitudeCutoff(metric_data=mc)
ac.compute_metric()

[array([0.0213507 , 0.17744927, 0.26078261, 0.0108107 , 0.27871942,
        0.02046109, 0.01176284, 0.01078261, 0.0103513 , 0.19047619])]

In [11]:
from spiketoolkit.validation.quality_metrics_new import compute_amplitude_cutoffs
recording_params_dict = {'apply_filter':False}

In [12]:
compute_amplitude_cutoffs(sorting=sort, recording=rec, recording_params_dict=recording_params_dict)

[array([0.0213507 , 0.17744927, 0.26078261, 0.0108107 , 0.27871942,
        0.02046109, 0.01176284, 0.01078261, 0.0103513 , 0.19047619])]